In [1]:
import os
import sys

root_dir = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
model_dir = os.path.join(root_dir, "model")
sys.path.append(root_dir)
sys.path.append(model_dir)

from collections import defaultdict
import itertools
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hoho_utils

os.environ['CUDA_LAUNCH_BLOCKING']='1' 

### 数据预处理

In [2]:
def suffix_time():
    current_time = time.time()
    local_time = time.localtime(current_time)
    time_str = time.strftime('%Y%m%d%H%M%S', local_time)
    return time_str

In [3]:
#test
hoho_utils.INDEXS_2_PIECES

'KARBNPCkarbnpc'

In [4]:
suffix = suffix_time()
suffix

'20230910110405'

In [5]:
data_dir = "../output/data"
file_list = os.listdir(data_dir)
file_count = len(file_list)
item_list = list()

print(f"file_count: {file_count}")

for i, filename in enumerate(file_list):
    file_path = os.path.join(data_dir, filename)
    with open(file_path, "r") as file:
        json_str = file.read()
        matches = json.loads(json_str)
        match_count = len(matches)

        for j, match in enumerate(matches):
#             print(f"match: {match}")
#             break

            base_id =  int(time.time() * 1000) + j
            for k, round in enumerate(match):
                match_id = f"{i}-{j}"
                round_id = f"{base_id}{k}"
                state = round[0]
                action = round[2]
                next_state = round[3]
                chapture_list = round[6]
                chaptures = ','.join(chapture_list)
                chapture_reward = round[7]

                win = 0
                done = round[5]
                if "K" in chapture_list:
                    win = -1
                    done = True
                elif "k" in chapture_list:
                    win = 1
                    done = True

                round_dict = {"match_id": match_id,
                              "round_id": round_id,
                              "state": state,
                              "action":action,
                              "next_state": next_state,
                              "done": done,
                              "chaptures": chaptures,
                              "chapture_reward": chapture_reward,
                              "win": win}
                item_list.append(round_dict)

                if win == 1 or win == -1:
                    break

file_count: 338


In [6]:
data = pd.DataFrame(item_list)

In [7]:
data.head(20)

,match_id,round_id,state,action,next_state,done,chaptures,chapture_reward,win
0,0-0,16943150457890,RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c...,f0e1,RNBAK1BNR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,False,N,-6.28,0
1,0-0,16943150457891,RNBAK1BcR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,h2g2,RNBAK1BcR/4A4/1C4C2/P1P1P1P1P/9/9/p1p1p1p1p/1c...,True,K,-4.92,-1
2,0-1,16943150457900,RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c...,f0e1,RNBAK1BNR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,False,N,-6.28,0
3,0-1,16943150457901,RNBAK1BcR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,h2g2,RNBAK1BcR/4A4/1C4C2/P1P1P1P1P/9/9/p1p1p1p1p/1c...,True,K,-4.92,-1
4,0-2,16943150457910,RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c...,f0e1,RNBAK1BNR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,False,N,-6.28,0
5,0-2,16943150457911,RNBAK1BcR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,h2g2,RNBAK1BcR/4A4/1C4C2/P1P1P1P1P/9/9/p1p1p1p1p/1c...,True,K,-4.92,-1
6,0-3,16943150457920,RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c...,f0e1,RNBAK1BNR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,False,N,-6.28,0
7,0-3,16943150457921,RNBAK1BcR/4A4/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c...,h2g2,RNBAK1BcR/4A4/1C4C2/P1P1P1P1P/9/9/p1p1p1p1p/1c...,True,K,-4.92,-1
8,0-4,16943150457930,RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c...,a0a1,1NBAKABNR/R8/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5...,False,N,-6.28,0
9,0-4,16943150457931,1cBAKABNR/R8/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/7c1...,b2b4,1cBAKABNR/R8/7C1/P1P1P1P1P/1C7/9/p1p1p1p1p/7c1...,False,,0.00,0


In [8]:
len(data[data["win"] == 0]), len(data[data["win"] == 1]), len(data[data["win"] == -1])

(24972, 1, 10052)

In [44]:
# data2 = data.groupby(["match_id"], as_index = False).agg({"action": lambda x: ".".join(x), "chapture_reward": sum})

### hoho_todo: 如果出现重复动作，如何分配？
data2 = data.groupby(["match_id"], as_index = False).agg({"action": lambda x: ",".join(x.unique()), "chapture_reward": sum, "win": min})
data2.rename(columns = {"action": "action_sequence", "chapture_reward": "chapture_value"}, inplace = True)

In [45]:
data2

,match_id,action_sequence,chapture_value,win
0,0-0,"f0e1,h2g2",-11.20,-1
1,0-1,"f0e1,h2g2",-11.20,-1
2,0-10,"e0e1,e1f1,i0i2,h2g2,a0a1,b2b5,i2i0,b5h5",-39.07,-1
3,0-11,"f0e1,h2g2",-11.20,-1
4,0-12,"e0e1,b2b4,h2e2,e2g2,c0a2,b0d1,e1e2,a2c4,e2f2,g...",-101.55,-1
...,...,...,...,...
10049,99-5,"f0e1,h2g2",-11.20,-1
10050,99-6,"e0e1,h2d2,d2d5,b2g2,g0i2,a3a4,a4a5,a0a5,b0c2,g...",-99.55,-1
10051,99-7,"f0e1,h2g2",-11.20,-1
10052,99-8,"f0e1,h2g2",-11.20,-1


In [46]:
data3 = data2.groupby("action_sequence", as_index = False).agg({"chapture_value": lambda x: x.mean(), "win": sum})
data3.rename(columns = {"win": "win_count"}, inplace = True)

In [47]:
data3

,action_sequence,chapture_value,win_count
0,"a0a1,a1c1,b2b8,c1f1,f1f8,c3c4,g0i2,e3e4,c0a2",-96.17,-1
1,"a0a1,b2b4,a1b1,h2d2,g0e2,b1f1,d0e1,e1f2",-37.61,-1
2,"a0a1,b2b4,a1b1,i3i4,b1e1,e1d1,h2b2,g3g4",-37.61,-1
3,"a0a1,b2b4,a1d1,d1a1,a1f1,h2h5,h0i2,f0e1,f1f6,f...",-105.37,-2
4,"a0a1,b2b4,a1d1,d1a1,f0e1,h2h4,h0i2,c0a2",-62.02,-1
...,...,...,...
805,"f0e1,g0e2,h2h7,b2b3,a0a1,a1a2",-49.45,-2
806,"f0e1,g0e2,h2h7,b2b9,h7e7,a3a4,a0a2,a2a0,g3g4",-87.71,-1
807,"f0e1,h2g2",-11.20,-7811
808,"f0e1,h2h5",-11.20,-363


In [30]:
actions_list = data3["action_sequence"].values.tolist()
list(map(lambda x: x.split(","), actions_list))

[['a0a1', 'a1c1', 'b2b8', 'c1f1', 'f1f8', 'c3c4', 'g0i2', 'e3e4', 'c0a2'],
 ['a0a1', 'b2b4', 'a1b1', 'h2d2', 'g0e2', 'b1f1', 'd0e1', 'e1f2'],
 ['a0a1', 'b2b4', 'a1b1', 'i3i4', 'b1e1', 'e1d1', 'h2b2', 'g3g4'],
 ['a0a1',
  'b2b4',
  'a1d1',
  'd1a1',
  'a1f1',
  'h2h5',
  'h0i2',
  'f0e1',
  'f1f6',
  'f6f0',
  'g0e2',
  'i0i1',
  'f0i0',
  'i1g1',
  'i3i4',
  'e0f0',
  'i2g1',
  'e2g0',
  'f0e0',
  'i0i3',
  'i3i0'],
 ['a0a1', 'b2b4', 'a1d1', 'd1a1', 'f0e1', 'h2h4', 'h0i2', 'c0a2'],
 ['a0a1', 'b2b4', 'a1d1', 'd1a1', 'f0e1', 'h2h4', 'i3i4', 'c0a2'],
 ['a0a1', 'b2b4', 'a1d1', 'd1a1', 'f0e1', 'h2h4', 'i3i4', 'i4i5'],
 ['a0a1',
  'b2b4',
  'a1d1',
  'e0e1',
  'd1b1',
  'e1e2',
  'b1b2',
  'g3g4',
  'b2b3',
  'h2f2',
  'b4b6',
  'f2h2',
  'e2d2'],
 ['a0a1', 'b2b4', 'a1d1', 'g0e2', 'd1b1', 'f0e1', 'i3i4', 'e2g0'],
 ['a0a1', 'b2b4', 'a1d1', 'h0g2', 'b4b1', 'b1b2', 'h2i2'],
 ['a0a1',
  'b2b4',
  'a1d1',
  'h0g2',
  'b4b1',
  'b1b2',
  'i0i2',
  'd1g1',
  'e0e1',
  'e1e2',
  'e2d2'],
 ['a0a1', '

In [18]:
# demo
test_list = ["e0e1", "e1f1", "i0i2", "h2g2", "a0a1", "b2b5", "i2i0", "b5h5"]
sorted(test_list)
# subsets(test_list)

['a0a1', 'b2b5', 'b5h5', 'e0e1', 'e1f1', 'h2g2', 'i0i2', 'i2i0']

In [55]:
test_list2 = ["e0e1", "e1f1", "e0e1"]
frozenset(test_list2)

frozenset({'e0e1', 'e1f1'})

In [63]:
from itertools import chain, combinations
from tqdm import tqdm
from collections import Counter, defaultdict


class SimplifiedShapleyAttributionModel:
    
    def powerset(self, x):
        s = list(x)
        return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))

    
    def _phi(self, channel_index):
        S_channel = [k for k in self.journeys.keys() if channel_index in k]
        score = 0
        print(f"Computing phi for channel {channel_index}...")
        for S in tqdm(S_channel):
            score += self.journeys[S] / len(S)
        print(f"Attribution score for channel {channel_index}: {score:.2f}")
        print()
        return score

    
    def attribute(self, df):
        actions_list = data3["action_sequence"].values.tolist()
        journeys_list = list(map(lambda x: x.split(","), actions_list))
        
        self.P = set(chain(*journeys_list))
#         print(f"self.P: {self.P}")
        print("Running Simplified Shapley Attribution Model...")
        print(f"Found {len(self.P)} unique channels!")
        
        print("Computing journey statistics...")
        value_dict = defaultdict(int)
        for i in range(len(df)):
            row = df.iloc[i]
            key = frozenset(row["action_sequence"].split(","))
#             value = row["win_count"]
            value = row["chapture_value"]
            value_dict[key] += value
        
        self.journeys = value_dict
        
#         print(f"self.journeys: {self.journeys}")
        
        print(f"Computing attributions...")
        print()
        return {j: self._phi(j) for j in self.P}

In [64]:
model = SimplifiedShapleyAttributionModel()
result = model.attribute(data3)

Running Simplified Shapley Attribution Model...
Found 670 unique channels!
Computing journey statistics...
Computing attributions...

Computing phi for channel f7f0...


100%|████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Attribution score for channel f7f0: -9.34

Computing phi for channel h0h4...


100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 1945.41it/s]


Attribution score for channel h0h4: -15.86

Computing phi for channel g6h6...


...

Attribution score for channel b8b2: -5.28

Computing phi for channel f7e7...


100%|████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Attribution score for channel f7e7: -5.92

Computing phi for channel e1f0...


100%|████████████████████████████████████████████████████| 9/9 [00:00<?, ?it/s]

Attribution score for channel e1f0: -48.15



In [65]:
total = 0
for k, v in result.items():
    print(f"Action '{k}': {v:.2f}")
    total += v

print(f"Total of attributed values: {total:.2f}")

Action 'f7f0': -9.34
Action 'h0h4': -15.86
Action 'g6h6': -11.21
Action 'd2f2': -45.74
Action 'a2a1': -115.29
Action 'f1f8': -10.69
Action 'g2e2': -149.75
Action 'd2e1': -26.40
Action 'e1e0': -655.08
Action 'b5h5': -4.88
Action 'i0g0': -38.69
Action 'f2e2': -135.61
Action 'g2h4': -23.71
Action 'b2b3': -462.83
Action 'e6d6': -6.61
Action 'c2f2': -76.89
Action 'd3g3': -3.82
Action 'b6b5': -6.72
Action 'h0h2': -14.84
Action 'c1a1': -26.59
Action 'd2a2': -56.49
Action 'd1e1': -102.42
Action 'b2b5': -217.20
Action 'e7c6': -8.73
Action 'h5b5': -8.56
Action 'h5h2': -80.23
Action 'a2a0': -120.16
Action 'h2a2': -32.82
Action 'h5h7': -10.79
Action 'e1g1': -11.84
Action 'h0i0': -15.82
Action 'f6f0': -14.63
Action 'd2c2': -37.42
Action 'h5i5': -103.62
Action 'c1g1': -13.42
Action 'h1e1': -14.39
Action 'b1b7': -8.05
Action 'c2a3': -2.82
Action 'a4c4': -21.50
Action 'b4b3': -67.28
Action 'b0b1': -36.41
Action 'c1a2': -9.00
Action 'd4d5': -5.96
Action 'f2c2': -72.02
Action 'h8h6': -23.59
Action 'd4d6

### 以下计算shapley value 方法复杂度很高，暂不采用

In [13]:
# 即生成列表List的所有组合
# 如List = [1,2,3]，则返回[], [1], [2], [3], [1,2], [1,3], [2,3], [1,2,3]
def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in itertools.combinations(List, i+1)]
    return PS

In [14]:
def subsets(s):
    if len(s) == 1:
        return s
    else:
        sub_channels = []
        for i in range(1, len(s) + 1):
            sub_channels.extend(map(list, itertools.combinations(s, i)))
    return list(map(",".join, map(sorted, sub_channels)))

In [16]:
def value_function(A, C_values):
    subsets_of_A = subsets(A)
    worth_of_A = 0
    for subset in subsets_of_A:
        print(f"subset: {subset}")
        if subset in C_values:
            worth_of_A += C_values[subset]
    return worth_of_A

In [17]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

In [23]:
#demo
data_demo = data3.head(2)

c_values = data_demo.set_index("action_sequence").to_dict()["chapture_value"]    
channels = []
for i in range(len(data_demo)):
    row = data_demo.iloc[i]
    actions = row["action_sequence"].split(",")
    channels.extend(actions)
    
channels = list(set(channels))
print(f"channels count: {len(channels)}")

power_channels = power_set(channels)
print(c_values)
    
# v_values = {}
# for A in power_channels:
#     v_values[','.join(sorted(A))] = value_function(A, c_values)
# print(f"v_values: {v_values}")


channels count: 16
{'a0a1,a1c1,b2b8,c1f1,f1f8,c3c4,g0i2,e3e4,c0a2': -96.17, 'a0a1,b2b4,a1b1,h2d2,g0e2,b1f1,d0e1,e1f2': -37.61}


In [21]:
v_values["a0a1,a1c1,b2b8,c1f1,f1f8,c3c4,g0i2,e3e4,c0a2"]

KeyError: 'a0a1,a1c1,b2b8,c1f1,f1f8,c3c4,g0i2,e3e4,c0a2'

In [28]:
def calculate_shapley(df, channel_name, conv_name):
    c_values = df.set_index(channel_name).to_dict()[conv_name]    
    df['channels'] = df[channel_name].apply(lambda x: x if len(x.split(",")) == 1 else np.nan)  # hoho: 这里可能会遗漏一些channel
    channels = list(df['channels'].dropna().unique())
    
#     print(f"power_set: {power_set(channels)}")
    
    v_values = {}
    for A in power_set(channels):
        v_values[','.join(sorted(A))] = v_function(A, c_values)
    n = len(channels)
    shapley_values = defaultdict(int)
    
    for channel in channels:
        for A in v_values.keys():
            if channel not in A.split(","):
                cardinal_A = len(A.split(","))  # 没有channel的长度
                A_with_channel = A.split(",")
                A_with_channel.append(channel)
                A_with_channel = ",".join(sorted(A_with_channel))
                weight = (factorial(cardinal_A) * factorial(n - cardinal_A - 1) / factorial(n)) # Weight = |S|!(n-|S|-1)!/n!
                contrib = (v_values[A_with_channel] - v_values[A]) # Marginal contribution = v(S U {i})-v(S)
                shapley_values[channel] += weight * contrib
                
        # Add the term corresponding to the empty set
        shapley_values[channel]+= v_values[channel] / n 
        
    return shapley_values

### 计算shapley value 方法复杂度很高，暂不采用